In [1]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
import os
import re
import nltk
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from openai import OpenAI
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = "/content/drive/MyDrive/Compartidos conmigo/Administracionsoftware/Modelos"
os.chdir(DATA_PATH)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
from openai import OpenAI
import openai

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
  raise ValueError("API key no encontrada en los secretos")

client = OpenAI(api_key=api_key)



---



In [ ]:
fake_df = pd.read_csv('Datos/Fake.csv')
true_df = pd.read_csv('Datos/True.csv')


fake_df['label'] = 1
true_df['label'] = 0

df = pd.concat([fake_df[['title', 'text', 'subject', 'label']], true_df[['title', 'text', 'subject', 'label']]], ignore_index=True)


# Verifiquemos la información del DataFrame:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   label    44898 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.4+ MB
None


In [ ]:
# Y veamos sus primeros registros:

# Para mostrar todo el texto sin cortes
print(df.head(5))

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  label  
0  Donald Trump just couldn t wish all Americans ...    News      1  
1  House Intelligence Committee Chairman Devin Nu...    News      1  
2  On Friday, it was revealed that former Milwauk...    News      1  
3  On Christmas day, Donald Trump announced that ...    News      1  
4  Pope Francis used his annual Christmas Day mes...    News      1  


In [ ]:
# Negaciones en inglés:
negwords = [ 'no', 'nor', 'not', 'ain', 'aren', "aren't", 'don', "don't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

my_stopwords = []

# Excluir palabras negativas de la lista de stopwords
for s_word in stopwords.words('english'):
    if s_word not in negwords:
        my_stopwords.append(s_word)

# Función para tokenizar y limpiar el texto
def tokenizar_y_limpiar(doc):
  # 1. Convierte el texto a minúsculas.
  doc = doc.lower()
  # 2. Conservar solamente los caracteres alfanuméricos, sin espacios, números o signos de puntuación.
  doc = re.sub(r'[^a-z]', ' ', doc)
  # 3. Eliminar mas de un espacio entre palabras y quitar espacios al principio y al final.
  doc = re.sub(r'\s+', ' ', doc).strip()
  # 4. Tokenizar el texto, eliminar las stopwords, y conservar solamente los tokens mayores a 1 carácter.
  tokens = [x for x in doc.split() if x not in my_stopwords and len(x) > 1]

  return tokens

# Función para limpiar el texto con stemming y lemmatization
def normalizar_texto(doc):
  ps = PorterStemmer()
  lemmatizer = WordNetLemmatizer()

  # Aplicando el stemming a cada token
  tokens = [ps.stem(x) for x in doc]
  # Lemmatizando cada token
  tokens = [lemmatizer.lemmatize(x) for x in tokens]

  return tokens

Xclean = [tokenizar_y_limpiar(text) for text in df.text]
Xclean = [normalizar_texto(x) for x in Xclean]
Y = df.label

In [ ]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x in Xclean[0:25]:
  print(x)

['donald', 'trump', 'couldn', 'wish', 'american', 'happi', 'new', 'year', 'leav', 'instead', 'give', 'shout', 'enemi', 'hater', 'dishonest', 'fake', 'news', 'medium', 'former', 'realiti', 'show', 'star', 'one', 'job', 'couldn', 'countri', 'rapidli', 'grow', 'stronger', 'smarter', 'want', 'wish', 'friend', 'support', 'enemi', 'hater', 'even', 'dishonest', 'fake', 'news', 'medium', 'happi', 'healthi', 'new', 'year', 'presid', 'angri', 'pant', 'tweet', 'great', 'year', 'america', 'countri', 'rapidli', 'grow', 'stronger', 'smarter', 'want', 'wish', 'friend', 'support', 'enemi', 'hater', 'even', 'dishonest', 'fake', 'news', 'medium', 'happi', 'healthi', 'new', 'year', 'great', 'year', 'america', 'donald', 'trump', 'realdonaldtrump', 'decemb', 'trump', 'tweet', 'went', 'welll', 'expect', 'kind', 'presid', 'send', 'new', 'year', 'greet', 'like', 'despic', 'petti', 'infantil', 'gibberish', 'trump', 'lack', 'decenc', 'won', 'even', 'allow', 'rise', 'gutter', 'long', 'enough', 'wish', 'american'

Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [ ]:
x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)

# verifiquemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 31428 31428
X,y Val: 6735 6735
X,y Test 6735 6735


Ahora, constriuiremos nuestro vocabulario

In [ ]:
mi_diccionario = Counter()

for palabra in range(len(x_train)):
    mi_diccionario.update(x_train[palabra])

print('Longitud original del vocabulario:', len(mi_diccionario))
# Conservar unicamente las palabras que aparecen más de 2 vez:
mi_diccionario = Counter({token: frecuencia for token, frecuencia in mi_diccionario.items() if frecuencia > 20})

Longitud original del vocabulario: 76502


In [ ]:
print('Longitud del vocabulario generado:', len(mi_diccionario))

print('10 palabras con mayor frecuencia:')
print(mi_diccionario.most_common(10))

In [ ]:
def filtrar_conjunto(conjunto, diccionario):
    resultado = []
    for review in conjunto:
        resultado.append([palabra for palabra in review if palabra in diccionario])
    return resultado

train_x = filtrar_conjunto(x_train, mi_diccionario)
val_x = filtrar_conjunto(x_val, mi_diccionario)
test_x = filtrar_conjunto(x_test, mi_diccionario)

In [ ]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

In [ ]:
from tqdm import tqdm   # pip install tqdm

vectores_embebidos = {}
tokens_utilizados = 0

# tqdm crea la barra; total=len(mi_diccionario) calcula el 100 %
for palabra in tqdm(mi_diccionario, total=len(mi_diccionario), desc="Generando embeddings"):
    response = client.embeddings.create(
        input=palabra,
        model="text-embedding-ada-002"
    )
    vectores_embebidos[palabra] = response.data[0].embedding
    tokens_utilizados += response.usage.total_tokens

print(f"Tokens totales consumidos: {tokens_utilizados:,}")


In [ ]:
print('Número de vectores generados:', len(vectores_embebidos))
# Verificamos el tamaño de los vectores generados
print('Tamaño de cada vector:', len(next(iter(vectores_embebidos.values()))))
# Verificamos el primer vector generado
print('Primer vector generado:', vectores_embebidos.get("star"))

# Guardamos los embeddings en un archivo JSON
with open('embeddings.json', 'w') as f:
    json.dump(vectores_embebidos, f)

Generamos los vectores embebidos a partir de los conjuntos de entrenamiento, validación y prueba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [ ]:
# Crear vector embebido por comentario, promediando los vectores de cada una de sus palabras
def generar_vectores_embebidos(comentarios, diccionario):
    vectores_embebidos = []
    for comentario in comentarios:
        vector_review = [0] * len(next(iter(diccionario.values())))
        for palabra in comentario:
            if palabra in diccionario:
                vector_review = [sum(x) for x in zip(vector_review, diccionario[palabra])]
        # Promediar el vector
        if len(comentario) > 0:
            vector_review = [x / len(comentario) for x in vector_review]
        vectores_embebidos.append(vector_review)
    return pd.DataFrame(np.array(vectores_embebidos))

trainEmb = generar_vectores_embebidos(train_x, vectores_embebidos)
valEmb = generar_vectores_embebidos(val_x, vectores_embebidos)
testEmb = generar_vectores_embebidos(test_x, vectores_embebidos)

In [ ]:
print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)


Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [ ]:
# REGRESIÓN LOGÍSTICA:

modelo_logistico = LogisticRegression(max_iter=1000, C=11, solver='saga', random_state=36)
modelo_logistico.fit(trainEmb, y_train)

# Predicciones en el conjunto de validación
predicciones_logistico = modelo_logistico.predict(valEmb)

# Cálculo de la exactitud y reporte
exactitud_logistico = accuracy_score(y_val, predicciones_logistico)
reporte_logistico = classification_report(y_val, predicciones_logistico)

print("Regresión Logística:")
print("Exactitud:", exactitud_logistico)
print("Reporte de clasificación:\n", reporte_logistico)